In [1]:
import pandas as pd 
import numpy as np
import os
import pyspark 
from pyspark.sql import SparkSession

In [2]:
#replace with correct directory where data files are located 
os.chdir(r'C:\Users\Nicho\gc-march-2022\data')

In [3]:
da = pd.read_csv('Indeed_DataAnalyst_2022.csv')
de = pd.read_csv('Indeed_DataEngineer_2022.csv')
ds = pd.read_csv('Indeed_DataScience_2022.csv')

In [4]:
da.info()
de.info()
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1155 non-null   int64 
 1   title        1155 non-null   object
 2   company      1153 non-null   object
 3   salary       388 non-null    object
 4   description  1153 non-null   object
 5   location     1153 non-null   object
dtypes: int64(1), object(5)
memory usage: 54.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1155 non-null   int64 
 1   title        1155 non-null   object
 2   company      1155 non-null   object
 3   salary       398 non-null    object
 4   description  1155 non-null   object
 5   location     1155 non-null   object
dtypes: int64(1), object(5)
memory usage: 54.3+ KB
<class 'pandas.core.frame.Data

There is a lot more rows in the engineering data than the other two sites. Only the salary column contains null values.

In [5]:
indeed_df = pd.concat([da,de,ds])
indeed_df.info()
indeed_df = indeed_df[['title','company','salary','description','location']]
indeed_df.drop_duplicates(inplace=True)
indeed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3463 entries, 0 to 1152
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   3463 non-null   int64 
 1   title        3463 non-null   object
 2   company      3461 non-null   object
 3   salary       1127 non-null   object
 4   description  3461 non-null   object
 5   location     3461 non-null   object
dtypes: int64(1), object(5)
memory usage: 189.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2348 entries, 0 to 1121
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2348 non-null   object
 1   company      2346 non-null   object
 2   salary       771 non-null    object
 3   description  2346 non-null   object
 4   location     2346 non-null   object
dtypes: object(5)
memory usage: 110.1+ KB


In [6]:
indeed_df['salary'] = indeed_df['salary'].str.lower()
indeed_df['description'] = indeed_df['description'].str.lower()
indeed_df['location'] = indeed_df['location'].str.lower()

In [7]:
indeed_df.head(5)

,title,company,salary,description,location
0,Data Science Analyst- Marketing,Indeed,"$100,000 - $140,000 a year",work across a wide range of data engineering t...,remote in texas+1 location
1,Data Analyst/Statistician,US Department of Veterans Affairs,"$51,980 - $103,518 a year",create processes for assessing quality and acc...,"hybrid remote in ann arbor, mi 48105"
2,Departmental Analyst 9-P11 - Data Analytics,State of Michigan,NaN,"and reporting, including biannual reports to t...","dimondale, mi"
3,CRM Data Analyst,Zeiss Industrial Quality Solutions,"$60,000 - $70,000 a year",responsible for maintaining sap/crm business p...,"wixom, mi 48393+1 location"
4,Data Analyst,Winwire Technologies,NaN,providing resource to add/delete/amend data wi...,remote


In [8]:
import re
indeed_df['salary'] = indeed_df['salary'].str.replace(',','')
indeed_df['salary'] = indeed_df['salary'].str.replace('-',' ')
indeed_df['salary'] = indeed_df['salary'].str.replace('$',' ')
indeed_df['salary'] = indeed_df['salary'].str.replace("[a-z]","")   
indeed_df['jobid'] = str(indeed_df['title']) + str(indeed_df['company']) + str(indeed_df['company'])


C:\Users\Nicho\AppData\Local\Temp/ipykernel_17068/3954269124.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  indeed_df['salary'] = indeed_df['salary'].str.replace('$',' ')
C:\Users\Nicho\AppData\Local\Temp/ipykernel_17068/3954269124.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['salary'] = indeed_df['salary'].str.replace("[a-z]","")


In [9]:
income_split = indeed_df['salary'].str.split(expand = True)
indeed_df['salary_lower_range'] =  income_split[0]
indeed_df['salary_upper_range'] =  income_split[1]
indeed_df.drop('salary',axis = 1,inplace=True)

In [10]:

indeed_df['company'].dropna(axis=0,inplace=True)
indeed_df['salary_lower_range'].fillna(0,inplace=True)
indeed_df['salary_upper_range'].fillna(0,inplace=True)
indeed_df['salary_lower_range'] = indeed_df['salary_lower_range'].astype(float)
indeed_df['salary_upper_range']  = indeed_df['salary_upper_range'].astype(float)
hourly_salaries = indeed_df[indeed_df['salary_lower_range'] < 100]
indeed_df = indeed_df[indeed_df['salary_lower_range'] > 100]

In [11]:
hourly_salaries['salary_lower_range']  = (hourly_salaries['salary_lower_range'] * 40) * 52
hourly_salaries['salary_upper_range'] = (hourly_salaries['salary_upper_range'] * 40) * 52
indeed_df['salary_lower_range'] = indeed_df['salary_lower_range'].astype(int)
indeed_df['salary_upper_range']  = indeed_df['salary_upper_range'].astype(int)

In [12]:
indeed_df = pd.concat([hourly_salaries,indeed_df])

In [13]:
indeed_df['is_remote'] = np.where(indeed_df['description'].str.contains('remote') | indeed_df['location'].str.contains('remote') ,1,0)
indeed_df[indeed_df['is_remote'] == 1]

,title,company,description,location,jobid,salary_lower_range,salary_upper_range,is_remote
4,Data Analyst,Winwire Technologies,providing resource to add/delete/amend data wi...,remote,0 Data Science Analyst...,0.0,0.0,1
5,Junior Data Analyst - DAL22-033349999999,"Evergreen Technologies, LLC.","experience with data wrangling, data cleaning ...","remote in detroit, mi+7 locations",0 Data Science Analyst...,0.0,0.0,1
6,Junior Data Analyst,Teamson,at least two years of experience as an analyst...,remote,0 Data Science Analyst...,0.0,0.0,1
8,"Senior Data Analyst, IT Remote",U.S. Medical Management,perform data cleansing to identify erroneous d...,"remote in troy, mi 48084",0 Data Science Analyst...,0.0,0.0,1
12,Data Analyst,Spokeo,we collect data from a wide variety of public ...,"remote in pasadena, ca 91101",0 Data Science Analyst...,0.0,0.0,1
...,...,...,...,...,...,...,...,...
1057,Lead Data Engineer,Morton,this person will touch every part of the data ...,remote,0 Data Science Analyst...,140000.0,180000.0,1
1069,Marketing Analyst (JO-2208-170877),Ledgent Technology,collect and analyze market and member data suc...,"hybrid remote in oceanside, ca 92054",0 Data Science Analyst...,70000.0,87000.0,1
1088,Software Engineering Manager,CyberCoders,"lead, manage, mentor and develop teams of soft...","remote in los angeles, ca 90066",0 Data Science Analyst...,170000.0,200000.0,1
1100,Software Engineering Manager,CyberCoders,"to meet business and consumer needs, we utiliz...","remote in los angeles, ca 90066",0 Data Science Analyst...,170000.0,200000.0,1


In [14]:
indeed_df['location'] = indeed_df['location'].str.replace('remote','')
indeed_df['location'] = indeed_df['location'].str.replace('in','')
indeed_df.groupby('is_remote').count()

,title,company,description,location,jobid,salary_lower_range,salary_upper_range
is_remote,,,,,,,
0,1033,1031,1031,1031,1033,1033,1033
1,1313,1313,1313,1313,1313,1313,1313


In [15]:
indeed_df['zipcode'] = indeed_df['location'].str.replace(r'\D+','')
indeed_df['zipcode'] = np.where(indeed_df['zipcode'] == '',0,indeed_df['zipcode'])
indeed_df['zipcode'] = indeed_df['zipcode'].fillna(0)
indeed_df['zipcode'] = indeed_df['zipcode'].astype('int')
indeed_df['zipcode'] = np.where(indeed_df['zipcode'] < 1000,0,indeed_df['zipcode'])
indeed_df.drop_duplicates(inplace = True)

C:\Users\Nicho\AppData\Local\Temp/ipykernel_17068/730417472.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['zipcode'] = indeed_df['location'].str.replace(r'\D+','')


In [16]:
indeed_df['location'] = indeed_df['location'].str.replace("[0-9]",'')

C:\Users\Nicho\AppData\Local\Temp/ipykernel_17068/1525615495.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['location'] = indeed_df['location'].str.replace("[0-9]",'')


In [17]:
indeed_df['location'] = indeed_df['location'].str.strip()
indeed_df['location'] = indeed_df['location'].str.split(',')

In [18]:
indeed_df['city'] = indeed_df['location'].str[0]
indeed_df['city'] = indeed_df['city'].str.replace("[0-9+()-]",' ')
indeed_df['state'] = indeed_df['location'].str[1]
indeed_df['state'] = indeed_df['state'].str.split()
indeed_df['state'] = indeed_df['state'].str[0]
indeed_df['state'] = indeed_df['state'].str.replace("[0-9+()-]",'')

C:\Users\Nicho\AppData\Local\Temp/ipykernel_17068/3846399863.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['city'] = indeed_df['city'].str.replace("[0-9+()-]",' ')
C:\Users\Nicho\AppData\Local\Temp/ipykernel_17068/3846399863.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['state'] = indeed_df['state'].str.replace("[0-9+()-]",'')


In [19]:
indeed_df['city'] = np.where(indeed_df['city'].str.contains('location'),'Remote',indeed_df['city'])
indeed_df.groupby('state').count()
indeed_df.groupby('city').count()           
indeed_df.drop('location',axis = 1,inplace=True)

In [20]:
indeed_df['title'] = indeed_df['title'].str.replace('[0-9]','')

C:\Users\Nicho\AppData\Local\Temp/ipykernel_17068/1637865698.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['title'] = indeed_df['title'].str.replace('[0-9]','')


In [21]:
indeed_df['job_type'] = np.where(indeed_df['title'].str.contains('analyst') | indeed_df['title'].str.contains('Analyst'),'Data Analyst','0')
indeed_df['job_type'] = np.where(indeed_df['title'].str.contains('engineer') | indeed_df['title'].str.contains('Engineer'),'Data Engineer',indeed_df['job_type'])
indeed_df['job_type'] = np.where(indeed_df['title'].str.contains('science') | indeed_df['title'].str.contains('Science'),'Data Scientist',indeed_df['job_type'])
indeed_df['job_type'] = np.where(indeed_df['job_type'] == '0','Other',indeed_df['job_type'])

In [22]:
indeed_df.drop_duplicates(inplace=True)
indeed_df.to_csv(r'C:\Users\Nicho\gc-march-2022\data\IndeedCleanedData.csv', index=False)